# Data preprocessing 

This jupyter notebook deals with data preprocessing for further analysis in the PyClone, SciClone and TitanCNA libraries. A prepared .vcf file and data obtained from the variant calling process are required for launch. For the preparation of variants, it is recommended to use the CNVpytor library, which ensures that the resulting data is immediately suitable for preprocessing without further modifications. When using other libraries, it is necessary to edit the data into the following format.



## Required libraries

Libraries necessary for the smooth running of the program (Dependencies excluded):

- [io](https://docs.python.org/3/library/io.html) - Core tools for working with streams
- [os](https://docs.python.org/3/library/os.html) - Miscellaneous operating system interfaces
- [pandas](https://pandas.pydata.org)  - Provides fast, flexible, and expressive data structures 
- [numpy](https://numpy.org) - Library used for working with arrays.
- [PyVCF](https://pyvcf.readthedocs.io/en/latest/) - A Variant Call Format Parser for Python
- [json](https://docs.python.org/3/library/json.html) - JSON encoder and decoder
- [pathlib](https://docs.python.org/3/library/pathlib.html) - Object-oriented filesystem paths
- [csv](https://docs.python.org/3/library/csv.html) - CSV File Reading and Writing
- [typing](https://docs.python.org/3/library/typing.html) - Support for type hints

For managing dependencies, we recommend using a package management system such as [Conda](https://docs.conda.io/en/latest/).



## CNVpytor / CNVnator

Both libraries are suitable for preparing variants as they work identically and return the same formats. CNVpytor serves as an extension of the CNVnator library for compatibility with the Python programming language.

[CNVpytor ](https://github.com/abyzovlab/CNVpytor)- Official documentation

[CNVpytor - Installer](https://anaconda.org/bioconda/cnvpytor) - Installation through conda

[CNVnator](https://github.com/abyzovlab/CNVnator) - Official documentation



### Format of variant calling results file (.tsv)

If you decide to use another library for the preparation of variants, you must follow the following guidelines. In general, this file must be in .tsv (Tab separated values) format. Values can be separated by tabs or number of whitespaces. The program does not recognizes other types of separators and will ignore them. **Columns required:**

- CNV type - "deletion" or "duplication"
- CNV region - chr:start-end
- CNV size - end-start
- CNV level - read depth normalized to 1
- e-val1 - e-value (p-value multiplied by genome size divided by bin size) calculated using t-test statistics between RD statistics in the region and global
- e-val2 - e-value (p-value multiplied by genome size divided by bin size) from the probability of RD values within the region to be in the tails of a gaussian distribution of binned RD
- e-val3 - same as e-val1 but for the middle of CNV
- e-val4 - same as e-val2 but for the middle of CNV
- q0 - fraction of reads mapped with q0 quality in call region
- pN - fraction of reference genome gaps (Ns) in call region
- dG - distance from closest large (>100bp) gap in reference genome


In [3]:
# Cell dealing with imports 
import io
import os
import pandas as pd
import numpy as np
import vcf
import json
from pathlib import Path
import mutation as mutation
from mutation import Mutation
import csv

In [ ]:
# Cell defining used custom functions

# Function that reads .vcf file and transforms it into dataframe.
def load_vcf_file(path):
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    return pd.read_csv(
        io.StringIO(u''.join(lines)),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})



def write_headers_as_json(vcf_reader, fn_out):
    """
    Once the vcf if loaded, put all the metadata from the header into
    a single json object and pretty print it to a file.
    Has no effect on the Reader but creates a new file
    """
    header_json = dict()

    header_json["metadata"] = vcf_reader.metadata
    header_json["infos"] =   vcf_reader.infos
    header_json["filters"] = vcf_reader.filters
    header_json["formats"] = vcf_reader.formats
    header_json["samples"] = vcf_reader.samples

    print("writing header info as json to : " + str(fn_out))
    with open(fn_out, 'w') as outfile:
        json.dump(header_json, outfile, indent=4)
    return

def extract_sample_id(input_filename):
    """
    infers a sample_id from the input filename of the vcf file.
    The result is the basename without the suffix (.vcf)
    """
    p = Path(input_filename)
    sample_id = (p.stem)
    return sample_id

def _construct_mutation_id(vcf_record):
        """
        The id of the mutation of the record has format
            chrom:pos:ref
        """
        chrom = str(vcf_record.CHROM)
        pos = str(vcf_record.POS)
        ref = str(vcf_record.REF)
        #mid = (chrom + ":" + pos + ":" + ref).decode('utf-8')
        mid = (chrom + ":" + pos)
        return mid



def write_pyclone_inputs(out_dirname, mutations):
    """
    write the mutations to a set of tsv files, one per sample, as described
    by the pyclone docs: https://github.com/Roth-Lab/pyclone
    """
    print("writing mutations as pyclone input format to: " + str(out_dirname))

    fieldnames = ['mutation_id', 'ref_counts', 'var_counts',
        'major_cn', 'minor_cn', 'normal_cn']

    #index the mutations by sample_id
    mutations_map = dict()
    for m in mutations:
        sample_id = m.sample_id
        if not sample_id in mutations_map:
            mutations_map[sample_id] = list()
        mutations_map[sample_id].append(m)

    for sample_id in mutations_map:
        out_fn = Path(out_dirname, (sample_id + '.tsv'))

        with open(str(out_fn), 'w') as csvfile:

            writer = csv.DictWriter(csvfile,
                fieldnames=fieldnames,
                delimiter='\t',
                quotechar='"',
                quoting=csv.QUOTE_MINIMAL,
                extrasaction='ignore')

            writer.writeheader()
            for m in mutations_map[sample_id]:
                md  = vars(m)
                #pyclone calls them 'var' not 'alt'
                md['var_counts'] = md['alt_counts']
                writer.writerow(md)
    return

To load the file, argument number 1, which was filled in at startup, is used.

In [ ]:
# Reading and making dataframe out of .vcf file
main_vcf = load_vcf_file('./DO52567.vcf')
main_vcf.head()

The next preprocessing step consists of filtering the loaded .vcf file. The file consists of a header and records. We can ignore the header as it is not necessary for our further analysis. For the subsequent search and analysis of the records, we select only the following columns from the main .vcf file: ID, POS, CHROM, 

In [2]:
# Filtering columns from main .vcf file.

# Creating empty dataframe
filtered_main_vcf = pd.DataFrame()

# Copying needed columns
filtered_main_vcf['POS'] = main_vcf['POS'].values
filtered_main_vcf['CHR'] = main_vcf['CHROM'].values

# Extracting needed information by splitting columns
filtered_main_vcf[['GENOTYPE', 'ALLELIC DEPTH', 'DEPTH', 'GENOTYPE QUALITY', 'PHRED-SCALED LIKELIHOODS']] = main_vcf.iloc[:, 9].str.split(":",expand=True,)

filtered_main_vcf.drop(['GENOTYPE QUALITY', 'PHRED-SCALED LIKELIHOODS'], axis=1)
filtered_main_vcf

NameError: name 'pd' is not defined

In [ ]:

# Extracting needed information by splitting columns
temp = main_vcf['INFO'].str.split(";",expand=True)
temp

# Extracting Minor and Major allele copy number 
filtered_main_vcf['ALLELIC FREQUENCY'] = temp[1].str[3:]
filtered_main_vcf['MINOR ALLELE COPY NUMBER'] = np.where(filtered_main_vcf['GENOTYPE'] == '0/1', 1, 0)
filtered_main_vcf['MAJOR ALLELE COPY NUMBER'] = np.where(filtered_main_vcf['GENOTYPE'] == '1/1', 2, 1)


chr1_vcf = filtered_main_vcf[filtered_main_vcf["CHR"] == 'chr1']
chr1_vcf

In [ ]:
cnvnator_df = pd.read_csv('test.tsv',  names=['CNV Type','CNV Region','CNV size','CNV level','e-val1', 'e-val2', 'e-val3','e-val4','q0','pN','dG'], delimiter=r"\s+")

cnvnator_df

ranges = cnvnator_df['CNV Region'].str.split(':',expand=True)
new = pd.DataFrame()
new[['start','end']] = ranges[1].str.split('-',expand=True)
new['COPY NUMBER'] = cnvnator_df['CNV level']
ranges = ranges[ranges[0] == '1']
new['start'] = new['start'].astype('int')
new['end'] = new['end'].astype('int')

new = new[new['start'] != 1]
chr1_vcf['POS'] = chr1_vcf['POS'].astype('int')


In [ ]:
chr1_vcf = chr1_vcf.assign(key = 1)
new = new.assign(key = 1)
merged = chr1_vcf.merge(new[['key', 'start', 'end', 'COPY NUMBER']], on='key')
filtered = merged[(merged['POS'] >= merged['start']) & (merged['POS'] <= merged['end'])]
filtered['COPY NUMBER'] = filtered['COPY NUMBER'].astype('float')



In [ ]:
filtered['COPY NUMBER'] = filtered['COPY NUMBER'].apply(np.round)
filtered.sort_values(by=['COPY NUMBER'])

In [ ]:
if (filtered['POS'].isin(chr1_vcf["POS"]).any(0)):
    chr1_vcf['COPY NUMBER'] = filtered['Copy Number']
else:
    chr1_vcf['COPY NUMBER'] = 2

chr1_vcf

In [ ]:

# Left join on 'position'
merged_df = chr1_vcf.merge(filtered, on='POS', how='left')
merged_df['COPY NUMBER'] = merged_df['COPY NUMBER'].fillna(2)
merged_df[['POS', 'MINOR ALLELE COPY NUMBER_x', 'MAJOR ALLELE COPY NUMBER_x', 'COPY NUMBER']]


In [ ]:
final_df = pd.DataFrame()
sciclone_df = pd.DataFrame()

final_df['mutation_id'] = 'chr1:' + merged_df['POS'].astype(str)

sciclone_df['chr'] = 'chr1'
sciclone_df['pos'] = merged_df['POS'].values

final_df[['ref_counts', 'var_counts', 'none']] = merged_df['ALLELIC DEPTH_x'].str.split(',',expand=True)
sciclone_df['ref_reads'] = final_df['ref_counts'].values
sciclone_df['var_reads'] = final_df['var_counts'].values
sciclone_df['vaf'] = final_df['var_counts'].astype(int) / (final_df['ref_counts'].astype(int) + final_df['var_counts'].astype(int))
final_df['minor_cn'] = merged_df['MINOR ALLELE COPY NUMBER_x'].values
final_df['major_cn'] = merged_df['MAJOR ALLELE COPY NUMBER_x'].values
final_df['normal_cn'] = merged_df['COPY NUMBER'].values
final_df['normal_cn'] = final_df['normal_cn'].astype('int')
final_df = final_df.drop('none', axis=1)
sciclone_df['chr'] = 'chr1'
sciclone_df.to_csv('sciclone2.tsv', sep="\t")
sciclone_df
sciclone_exclude = pd.DataFrame(columns=['chromosomes', 'start', 'end'])
sciclone_exclude['chromosomes'] = 'chr1'
sciclone_exclude['start'] = 1
sciclone_exclude['end'] = 5

df2 = {'chromosomes': 'chr1', 'start': 1, 'end': 5}
sciclone_exclude = sciclone_exclude.append(df2, ignore_index = True)

sciclone_exclude.to_csv('sciclone_exclude.tsv', sep="\t")

sciclone_exclude

In [ ]:
sciclone_df

In [ ]:
sciclone_df2 = pd.DataFrame()
sciclone_dffinal = pd.DataFrame()

sciclone_df2[['start','end']] = ranges[1].str.split('-',expand=True)
sciclone_df2['start'] = sciclone_df2['start'].astype('int')
sciclone_df2['end'] = sciclone_df2['end'].astype('int')
sciclone_df2['segment_mean'] = cnvnator_df['CNV level']
sciclone_df2['segment_mean'] = sciclone_df2['segment_mean'].apply(np.round)
sciclone_df2['chr'] = "chr1"
sciclone_df2
sciclone_dffinal['chr'] = sciclone_df2['chr'].values
sciclone_dffinal['start'] = sciclone_df2['start'].values
sciclone_dffinal['end'] = sciclone_df2['end'].values
sciclone_dffinal['segment_mean'] = sciclone_df2['segment_mean'].values

sciclone_dffinal.to_csv('sciclone.tsv', sep="\t")
sciclone_dffinal

In [ ]:
final_df.to_csv('example.tsv', sep="\t")

In [ ]:
!PyClone run_analysis_pipeline --in_files example_subset.tsv --working_dir ./finalTest

In [ ]:
vcf_reader = vcf.Reader
vcf_reader = vcf.Reader(open("DO52567.vcf", 'r'))

write_headers_as_json(vcf_reader, 'headers.json')

sample_id = extract_sample_id("DO52567.vcf")


mutation_list = []
#print(vcf_reader.metadata)
tumor_samples = vcf_reader.metadata["tumor_sample"]
for rec in vcf_reader:
    for sample in tumor_samples:
        mutation_id = _construct_mutation_id(rec)
        tumor_call = rec.genotype(sample)
        call_data = tumor_call.data
        print(call_data)
        counts = call_data.AD
        ref_counts = counts[0]
        alt_counts = counts[1]
        mutation_list.append(Mutation(sample, mutation_id, ref_counts, alt_counts))

In [ ]:
final_df[final_df['POS'] == 248897833].minor_cn
minor_cn = final_df[final_df['POS'] == rec.POS].minor_cn
major_cn = final_df[final_df['POS'] == rec.POS].major_cn
normal_cn = final_df[final_df['POS'] == rec.POS].normal_cn

## Creating input data for PyClone

In [ ]:



write_pyclone_inputs("output", mutation_list)



## Creating input data for SciClone

## Creating input data for TitanCNA

In [ ]:
subset_tsv = pd.read_csv('output/DO52567.tsv', sep='\t')
subset_tsv = subset_tsv.loc[0:200]
subset_tsv
subset_tsv.to_csv('example_subset.tsv', sep="\t")
subset_tsv

## SciClone

In [ ]:
subset_sciclone = pd.read_csv('sciclone2.tsv', sep='\t')
subset_sciclone.drop(columns=subset_sciclone.columns[0], axis=1, inplace=True)
subset_sciclone = subset_sciclone.loc[0:200]
subset_sciclone.to_csv('sciclone_subset.tsv', sep="\t")
subset_sciclone

In [ ]:
mask = chr1_vcf['POS'].isin(range(new['start'], new['end'] + 1))

## TitanCNA

To prepare the necessary files for the TitanCNA library, we need to prepare two specific file types - .het (heterozygosity file) and .cn (copy number  file). Both of these files are required by the TitanCNA library in following formats (Observing the order and nomenclature of the columns is recommended):

#### **Heterozygosity file**

- **chr -** Chromosome name
- **posn -** Mutation position
- **ref -** Refference allele
- **refCount -** Refference allele count
- **Nref -** Alternative Allele
- **NrefCount -** Alternative Allele count



#### Copy number file

- **chr - **Chromosome name
- **start -** Start of the segment
- **end -** End of the segment
- **logR -** 



In [ ]:
# Creation of .het file
titnacna_df = pd.DataFrame()

titnacna_df['chr'] = sciclone_df['chr'].values
titnacna_df['posn'] = sciclone_df['pos'].values
titnacna_df['ref'] = main_vcf['REF']
titnacna_df['refCount'] = sciclone_df['ref_reads'].values
titnacna_df['Nref'] = main_vcf['ALT']
titnacna_df['NrefCount'] = sciclone_df['var_reads'].values
titnacna_df['chr'] = 1
titnacna_df.to_csv('input_files/TitanCNA/titancna.het', sep="\t", index=False)




In [1]:
# Creation of .cn file
titnacna_df2 = pd.DataFrame()
titnacna_df2['chr'] = sciclone_df2['chr'].values
titnacna_df2['chr'] = 1
titnacna_df2['start'] = sciclone_df2['start'].values
titnacna_df2['end'] = sciclone_df2['end'].values
titnacna_df2['logR'] =  sciclone_df2['segment_mean'].values

titnacna_df2.to_csv('input_files/TitanCNA/titancna.cn', sep="\t", index=False)

NameError: name 'pd' is not defined

In [ ]:
mask = pd.DataFrame()
for index, row in chr1_vcf.iterrows():
    for index2, row2 in new.iterrows():
        if ((row['POS'] >= row2['start']) & (row['POS'] <= row2['end'])):
            mask.append(row)

In [ ]:
df1 = df1.assign(key = 1)
df2 = df2.assign(key = 1)
merged = df1.merge(df2[['key', 'start', 'end']], on='key')
filtered = merged[(merged['position'] >= merged['start']) & (merged['position'] <= merged['end'])]
filtered

In [ ]:
mask1 = (chr1_vcf['POS'] >= new['start'])
mask2 = (chr1_vcf['POS'] <= new['end'])
chr1_vcf = chr1_vcf[~(mask1 & mask2)]


In [ ]:
def extract_frequency_from_variant(variant):
    # Split the variant string into its components
    components = variant.strip().split("\t")

    # Extract the frequency information from the INFO field
    info = components[7]
    frequency = None
    for field in info.split(";"):
        if field.startswith("AF="):
            frequency = float(field[3:])
            break
    
    # Return the frequency information
    return frequency
    
# Open the VCF file and read the contents into a list of variants
with open("DO52567.vcf", "r") as f:
    variants = f.readlines()

# Extract the frequency information from each variant and store it in a separate list
frequencies = []
for variant in variants:
    frequency = extract_frequency_from_variant(variant) # Custom function to extract frequency information from the variant
    frequencies.append(frequency)

# Sort the variants based on frequency
sorted_variants = [variant for _, variant in sorted(zip(frequencies, variants))]

# Write the sorted variants to a new VCF file
with open("output.vcf", "w") as f:
    f.writelines(sorted_variants)

In [ ]:
# Create the data frame with ranges
ranges_df = pd.DataFrame({'start': [100, 200, 300],
                         'end': [150, 250, 350]})

# Create the data frame with numbers
numbers_df = pd.DataFrame({'number': [120, 230, 320, 390]})

# Define a custom function to check if the number is in the range
def is_in_range(num, start, end):
    return (num >= start) & (num <= end)

# Apply the custom function to each row of the numbers data frame using the ranges data frame
result = parsed_df.apply(lambda row: new.apply(lambda x: is_in_range(row['POS'], x['start'], x['end']), axis=1).any(), axis=1)

# The result is a series with a Boolean value for each number indicating if it is in any of the ranges
print(result)